### Objectif : Créer une API RESTful pour gérer des articles de blog.
#Exercice :
#Utilisez Flask pour créer une API avec les fonctionnalités suivantes :
#Récupérer tous les articles (GET /articles).
#Ajouter un nouvel article (POST /articles).
#Mettre à jour un article (PUT /articles/<id>).
#Supprimer un article (DELETE /articles/<id>).
#Les articles doivent être stockés dans une base de données SQLite.

In [1]:
from flask import Flask, jsonify, request
import sqlite3

app = Flask(__name__)
DATABASE = "blog.db"

# Fonction pour se connecter à la base de données
def get_db_connection():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row  # Pour un accès par clé comme un dictionnaire
    return conn

# Création de la table articles
def init_db():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titre TEXT NOT NULL,
            contenu TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# Récupérer tous les articles (GET /articles)
@app.route('/articles', methods=['GET'])
def get_articles():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM articles')
    articles = cursor.fetchall()
    conn.close()

    return jsonify([dict(article) for article in articles])

# Ajouter un nouvel article (POST /articles)
@app.route('/articles', methods=['POST'])
def create_article():
    data = request.json
    titre = data.get('titre')
    contenu = data.get('contenu')

    if not titre or not contenu:
        return jsonify({"message": "Titre et contenu sont requis."}), 400

    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('INSERT INTO articles (titre, contenu) VALUES (?, ?)', (titre, contenu))
    conn.commit()
    conn.close()

    return jsonify({"message": "Article créé avec succès."}), 201

# Mettre à jour un article (PUT /articles/<id>)
@app.route('/articles/<int:id>', methods=['PUT'])
def update_article(id):
    data = request.json
    titre = data.get('titre')
    contenu = data.get('contenu')

    if not titre or not contenu:
        return jsonify({"message": "Titre et contenu sont requis."}), 400

    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM articles WHERE id = ?', (id,))
    article = cursor.fetchone()

    if article is None:
        conn.close()
        return jsonify({"message": "Article non trouvé."}), 404

    cursor.execute('UPDATE articles SET titre = ?, contenu = ? WHERE id = ?', (titre, contenu, id))
    conn.commit()
    conn.close()

    return jsonify({"message": "Article mis à jour avec succès."})

# Supprimer un article (DELETE /articles/<id>)
@app.route('/articles/<int:id>', methods=['DELETE'])
def delete_article(id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM articles WHERE id = ?', (id,))
    article = cursor.fetchone()

    if article is None:
        conn.close()
        return jsonify({"message": "Article non trouvé."}), 404

    cursor.execute('DELETE FROM articles WHERE id = ?', (id,))
    conn.commit()
    conn.close()

    return jsonify({"message": "Article supprimé avec succès."})

if __name__ == '__main__':
    init_db()  # Initialiser la base de données au lancement
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packag

SystemExit: 1

/Users/sciarbouche/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
